In [1]:
import towhee
import os
import glob
from PIL import Image
import pandas as pd
import numpy as np
import unicodedata

c:\Users\Admin\anaconda3\envs\env4ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vietnamese Engslish Translate

In [2]:
dict_map = {
    "òa": "oà","Òa": "Oà","ÒA": "OÀ","óa": "oá","Óa": "Oá","ÓA": "OÁ","ỏa": "oả","Ỏa": "Oả","ỎA": "OẢ","õa": "oã",
    "Õa": "Oã","ÕA": "OÃ","ọa": "oạ","Ọa": "Oạ","ỌA": "OẠ","òe": "oè","Òe": "Oè","ÒE": "OÈ","óe": "oé","Óe": "Oé",
    "ÓE": "OÉ","ỏe": "oẻ","Ỏe": "Oẻ","ỎE": "OẺ","õe": "oẽ","Õe": "Oẽ","ÕE": "OẼ","ọe": "oẹ","Ọe": "Oẹ","ỌE": "OẸ",
    "ùy": "uỳ","Ùy": "Uỳ","ÙY": "UỲ","úy": "uý","Úy": "Uý","ÚY": "UÝ","ủy": "uỷ","Ủy": "Uỷ","ỦY": "UỶ","ũy": "uỹ",
    "Ũy": "Uỹ","ŨY": "UỸ","ụy": "uỵ","Ụy": "Uỵ","ỤY": "UỴ",
    }

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("../AIChallenge_data/ViEnTranslate/", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("../AIChallenge_data/ViEnTranslate/")

def translate_vi2en(vi_text: str) -> str:
    for i, j in dict_map.items():
        vi_text = vi_text.replace(i, j)
    input_ids = tokenizer_vi2en(vi_text, return_tensors="pt").input_ids
    output_ids = model_vi2en.generate(
        input_ids,
        do_sample=True,
        top_k=100,
        top_p=0.8,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
    )
    en_text = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    en_text = " ".join(en_text)
    en_text = strip_accents(en_text)
    en_text = en_text.replace("\\","")
    return en_text

# vi_text = "Cô cho biết: trước giờ tôi không đến phòng tập công cộng, mà tập cùng giáo viên Yoga riêng hoặc tự tập ở nhà. Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn."
# print(translate_vi2en(vi_text))

# Text-image inference

In [4]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

In [5]:
%cd ../AIChallenge_data/

d:\GitHub\AIChallenge_data


In [6]:
import pandas as pd

df_keyframes = pd.read_csv('key_frame_df.csv',index_col=0)

import cv2
from towhee._types.image import Image

id_img = df_keyframes.set_index('id')['img_path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        # print(re)
        path = id_img[re.id]
        imgs.append(path)
    return imgs

In [8]:
query_dir = "D:\GitHub\AIChallenge_data\query-pack-0\query-pack-0-3"
en_query_dir = os.path.join(os.path.dirname(query_dir),"en_"+ query_dir.split("\\")[-1])
try:
    os.mkdir(en_query_dir)
except:
    pass
list_queries_file = os.listdir(query_dir)
en_queries = []
for query in list_queries_file:
    with open(os.path.join(query_dir,query),"r",encoding='utf8') as f:
        with open(os.path.join(en_query_dir,"en_" + query),"w",encoding='utf8') as f_out:
            read_line = f.readline()
            print("{}: {}\n".format(query,read_line))
            vi2en_text = translate_vi2en(read_line)
            f_out.write(vi2en_text)
            en_queries.append(vi2en_text)

print("---------------")
print(en_queries)

query-3.txt: Một sở thú tại Trung Quốc với nhiều loài động vật khác nhau. Trong khung hình có 2 con hà mã. Một trong 2 đang uống nước. Ngoài hà mã, sở thú còn có voi và gấu trúc.

query-4.txt: Một bức hình của Tổng thống Mỹ Joe Biden. Tổng thống Mỹ đang đeo kính đen. Đó là trang nhất của một bài báo. Tiêu đề tiếng Việt ghi: Mỹ có thể sẽ miễn thuế pin năng lượng mặt trời từ Việt Nam. Đó là một bài báo từ hãng tin Reuters.

query-5.txt: Người nghệ nhân đang tô màu cho chiếc mặt nạ một cách tỉ mỉ. Xung quanh ông là rất nhiều những chiếc mặt nạ. Người nghệ nhân đi đôi dép tổ ong rất giản dị. Loại mặt nạ này được gọi là mặt nạ giấy bồi Trung thu.

---------------
['A zoo in China with many different animals. In the frame there are 2 hippos. One of the 2 is drinking water. In addition to hippos, the zoo also has elephants and raccoons.', 'A picture of U.S. President Joe Biden. The U.S. President is wearing black glasses. It was the front page of an article. The Vietnamese title read: The U.S

In [9]:
collection = Collection("text_image_challenge_search")
(
      towhee.dc['text'](en_queries)
        .image_text_embedding.clip['text', 'vec'](model_name='clip_vit_b32', modality='text')
        .tensor_normalize['vec','vec']()
        .milvus_search['vec', 'result'](collection=collection, limit=100)
        .runas_op['result', 'result_img'](func=read_images)
        .select['text','result_img']()
        .to_csv("test.csv")
)

In [10]:
from ast import literal_eval
df = pd.read_csv("test.csv")
df.result_img =df.result_img.apply(literal_eval)

result_query_dir = os.path.join(os.path.dirname(query_dir),"result")
try:
    os.mkdir(result_query_dir)
except:
    pass

for i in range(len(list_queries_file)):
    with open(os.path.join(result_query_dir,"{}.csv".format(list_queries_file[i].split(".")[0].replace("en_",""))),"w") as f:
        list_img = df["result_img"][i]
        for img in list_img:
            video_id = img.split("\\")[-2]
            false_frame = img.split("\\")[-1].split(".")[0]
            real_frame = df_keyframes[df_keyframes.false_frame == int(false_frame)][video_id==df_keyframes.video_id].frame.values[0]
            f.write(video_id + ".mp4, " + str(real_frame) + '\n')